In [1]:
!pip install transformers[torch] sacrebleu evaluate
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install rouge-metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731

## Загружаем данные

В тренировочной выборке 4411 примера, в валидационной 111 пример, в тестовой 992 примера.

In [ ]:
import json
from sklearn.model_selection import train_test_split

data = json.load(open('russian_dataset_rewrite_checked.json'))
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.1, random_state=42)

In [ ]:
len(train_data), len(test_data), len(val_data)

(4411, 111, 992)

In [ ]:
# with open('train_data_rewrite.json', 'w', encoding='utf-8') as file:
#     json.dump({'data': train_data}, file, ensure_ascii=False, indent=2)

# with open('test_data_rewrite.json', 'w', encoding='utf-8') as file:
#     json.dump({'data': test_data}, file, ensure_ascii=False, indent=2)

# with open('val_data_rewrite.json', 'w', encoding='utf-8') as file:
#     json.dump({'data': val_data}, file, ensure_ascii=False, indent=2)

In [2]:
from datasets import load_dataset

dataset = load_dataset('json', data_files={'train': 'train_data_rewrite.json',
                                           'test': 'test_data_rewrite.json',
                                           'val': 'val_data_rewrite.json'},
                       field='data')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

## Предобработка данных

In [3]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained('ai-forever/ruT5-base',eos_token='</s>')
model = T5ForConditionalGeneration.from_pretrained('ai-forever/ruT5-base')

def preprocess_function(examples):
    # prompt with context and real phrase
    phrases = [example for example in examples["Phrase"]]
    contexts = ['\n,'.join([f'<SC5>"{c}"</SC5>'  for c in example[-2:]]) for example in examples["History"]]
    inputs = [f'''
Перепиши данную фразу согласно данному контексту, так чтобы фраза стала контекстно независимой. Исключи из фразы эллипсис и кореференцию.
КОНТЕКСТ:
{context}
Фраза:
{phrases[i]}
Перефраз: <extra_id_0>
''' for i, context in enumerate(contexts)]
    # rewrites
    targets = [example for example in examples["Rewrite"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model='ai-forever/ruT5-base',
                                       pad_to_multiple_of=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

Map:   0%|          | 0/4411 [00:00<?, ? examples/s]

Map:   0%|          | 0/992 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

## Метрики

In [4]:
import evaluate
from rouge_metric import PyRouge

bleu = evaluate.load("sacrebleu")
rouge = PyRouge(rouge_n=(4), skip_gap=4)

import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = bleu.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    rouge_score = rouge.evaluate(decoded_preds, decoded_labels)
    result['rouge_l'] = rouge_score['rouge-l']['f']
    result['rouge_2'] = rouge_score['rouge-2']['f']

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

## Train

In [5]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

from peft import LoraConfig, get_peft_model
from peft.utils.peft_types import TaskType

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

model = get_peft_model(model, config)

In [6]:
new_model = 'my_rewrite_base_t5'

training_args = Seq2SeqTrainingArguments(
    output_dir=new_model,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu,Rouge L,Rouge 2,Gen Len
0,No log,4.542775,1.052100,0.044300,0.006600,19.000000
1,No log,2.953781,6.566600,0.143700,0.058500,19.000000
2,No log,2.220399,8.372500,0.233800,0.140400,10.499000
4,No log,1.909067,23.639600,0.440800,0.333700,13.365900
5,No log,1.811206,29.085600,0.511800,0.398200,13.663300
6,No log,1.770657,31.454100,0.536900,0.422800,13.871000
8,No log,1.705003,33.909900,0.562400,0.447200,14.374000
9,No log,1.706100,34.088200,0.566900,0.450900,14.361900
10,No log,1.674719,34.381100,0.569100,0.454200,14.405200
12,No log,1.648776,34.802800,0.573100,0.457600,14.520200


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
trainer.evaluate(tokenized_dataset["val"])

{'eval_loss': nan,
 'eval_bleu': 30.9459,
 'eval_gen_len': 19.0,
 'eval_runtime': 899.0242,
 'eval_samples_per_second': 0.944,
 'eval_steps_per_second': 0.944,
 'epoch': 1.9974470257850396}

In [7]:
import torch
from tqdm.notebook import tqdm

In [8]:
raw_test_results_new = []
model = model.to('cuda')

for i, encoding in tqdm(enumerate(tokenized_dataset['test']['input_ids'])):
    input_ids=torch.tensor([encoding]).to('cuda')
    out = model.generate(inputs=input_ids,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=150,
                         num_beams=2,
                         repetition_penalty=3.,
                         length_penalty=3.0,
                         early_stopping=True)
    out = tokenizer.decode(out[0][1:],
                           skip_special_tokens=True,
                           clean_up_tokenization_spaces=True)
    raw_test_results_new.append((tokenized_dataset['test'][i]['Utt_ID_hash'],
                             tokenized_dataset['test'][i]['Dia_ID_hash'],
                             out))

0it [00:00, ?it/s]

In [9]:
import pandas as pd

raw_test_results = pd.DataFrame(raw_test_results_new, columns=['Utt_ID_hash', 'Dia_ID_hash', 'model_out_raw'])
raw_test_results.head()

,Utt_ID_hash,Dia_ID_hash,model_out_raw
0,utt_1c9499ef,dia_8b1e5abb,То обязательно обращусь к вам.
1,utt_ed502a14,dia_3db27511,"Разные сериалы, главное, чтоб не нервные, а то..."
2,utt_f9015b3f,dia_1684762c,"Круто, я умею готовить! А что ты умеешь готовить?"
3,utt_87197d1a,dia_5dd50bca,Я бы побалуюсь на велосипеде - это более полез...
4,utt_7d293283,dia_043da1df,Секретарь. Хотя готовить всегда нравилось боль...


In [10]:
raw_test_results['Phrase'] = tokenized_dataset['test']['Phrase']
raw_test_results['Rewrite'] = tokenized_dataset['test']['Rewrite']

**Bleu**

In [12]:
results = bleu.compute(predictions=raw_test_results.model_out_raw.values.tolist(), references=raw_test_results.Rewrite.values.tolist())
print(results['score'])

34.198896532456935


In [13]:
results = bleu.compute(predictions=raw_test_results.model_out_raw.values.tolist(), references=raw_test_results.Phrase.values.tolist())
print(results['score'])

44.197992352763634


**Rouge**

In [14]:
scores = rouge.evaluate(raw_test_results.model_out_raw.values.tolist(), raw_test_results.Rewrite.apply(lambda x: [x]).values.tolist())
for k in scores:
    print(k, scores[k]['f'])

rouge-1 0.5366912662619522
rouge-2 0.3951366135806867
rouge-3 0.30228331149144216
rouge-4 0.22820169880779465
rouge-l 0.5267900789514197


In [15]:
scores = rouge.evaluate(raw_test_results.model_out_raw.values.tolist(), raw_test_results.Phrase.apply(lambda x: [x]).values.tolist())
for k in scores:
    print(k, scores[k]['f'])

rouge-1 0.5713822560579963
rouge-2 0.4670942993600828
rouge-3 0.3895367676252729
rouge-4 0.30671728100990236
rouge-l 0.5662813278217507


**Restoration F1**

In [16]:
import numpy as np

class RestorationFScore:

    def __init__(self, tokenizer, n_gram: int=2):
        self.n_gram = n_gram
        self.tokenizer = tokenizer

    def preprocess(self, sents):
        for sent in sents:
            sent_tokenize = self.tokenizer(sent)['input_ids']
            yield [tuple(sent_tokenize[i:i+self.n_gram]) for i, _ in enumerate(sent_tokenize)]

    def _itereval(self):
        for i, predictions in enumerate(self.predictions):
            restored_ngrams = set(predictions).difference(self.references[i])
            ngrams_in_ref = set(self.rewrites[i]).difference(self.references[i])
            interagree = ngrams_in_ref.intersection(restored_ngrams)
            if len(restored_ngrams):
                precision = len(interagree) / len(restored_ngrams)
            else:
                precision = 0.
            if len(ngrams_in_ref):
                recall = len(interagree) / len(ngrams_in_ref)
            else:
                recall = 0.
            if precision or recall:
                yield 2 * ((precision * recall) / (precision + recall))
            else:
                yield 0.

    def evaluate(self, predictions: list,
                 references: list, rewrites: list):
        self.predictions = [p for p in self.preprocess(predictions)]
        self.references = [p for p in self.preprocess(references)]
        self.rewrites = [p for p in self.preprocess(rewrites)]
        return np.mean(list(self._itereval()))

In [17]:
rf_score = RestorationFScore(tokenizer, 1)
rf_score.evaluate(predictions=raw_test_results.model_out_raw.values.tolist(),
                 references=raw_test_results.Phrase.values.tolist(),
                 rewrites=raw_test_results.Rewrite.values.tolist())

0.19888681840290004

In [18]:
rf_score = RestorationFScore(tokenizer, 2)
rf_score.evaluate(predictions=raw_test_results.model_out_raw.values.tolist(),
                 references=raw_test_results.Phrase.values.tolist(),
                 rewrites=raw_test_results.Rewrite.values.tolist())

0.13285988048990177

In [19]:
rf_score = RestorationFScore(tokenizer, 3)
rf_score.evaluate(predictions=raw_test_results.model_out_raw.values.tolist(),
                 references=raw_test_results.Phrase.values.tolist(),
                 rewrites=raw_test_results.Rewrite.values.tolist())

0.10622029450283109

In [20]:
rf_score = RestorationFScore(tokenizer, 4)
rf_score.evaluate(predictions=raw_test_results.model_out_raw.values.tolist(),
                 references=raw_test_results.Phrase.values.tolist(),
                 rewrites=raw_test_results.Rewrite.values.tolist())

0.093716832458188

## Сохранение модели

In [21]:
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


('my_rewrite_base_t5/tokenizer_config.json',
 'my_rewrite_base_t5/special_tokens_map.json',
 'my_rewrite_base_t5/spiece.model',
 'my_rewrite_base_t5/added_tokens.json')

In [23]:
!zip -r /content/my_rewrite_base_t5_new.zip /content/my_rewrite_base_t5

  adding: content/my_rewrite_base_t5/ (stored 0%)
  adding: content/my_rewrite_base_t5/adapter_model.safetensors (deflated 7%)
  adding: content/my_rewrite_base_t5/added_tokens.json (deflated 83%)
  adding: content/my_rewrite_base_t5/adapter_config.json (deflated 51%)
  adding: content/my_rewrite_base_t5/special_tokens_map.json (deflated 85%)
  adding: content/my_rewrite_base_t5/spiece.model (deflated 56%)
  adding: content/my_rewrite_base_t5/README.md (deflated 66%)
  adding: content/my_rewrite_base_t5/runs/ (stored 0%)
  adding: content/my_rewrite_base_t5/runs/Jun05_17-25-08_ab1df38cde96/ (stored 0%)
  adding: content/my_rewrite_base_t5/runs/Jun05_17-25-08_ab1df38cde96/events.out.tfevents.1717608309.ab1df38cde96.271.0 (deflated 65%)
  adding: content/my_rewrite_base_t5/tokenizer_config.json (deflated 94%)
